<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.1: Generators: Parameters
**Prev: [ChiselTest (was chisel-testers2)](2.6_chiseltest.ipynb)**<br>
**Next: [Generators: Collections](3.2_collections.ipynb)**

## Motivation
Chisel 모듈이 코드 생성기가 되려면 생성기가 작업을 어떻게 수행해야 하는지 알려주는 무언가가 있어야 합니다. 이 섹션에서는 모듈 매개변수화, 다양한 방법론 및 스칼라 언어 기능에 대해 논의합니다. 매개변수 전달 구현의 풍부함은 생성된 회로의 풍부함에 정비례합니다. 매개변수는 유용한 기본값을 제공하고 설정하기 쉽고 불법적이거나 무의미한 값으로부터 보호해야 합니다. 더 복잡한 시스템의 경우 다른 모듈 사용에 부주의하게 영향을 미치지 않는 방식으로 로컬로 재정의할 수 있는 경우 매우 유용합니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Parameter Passing
Chisel은 하드웨어 생성기를 작성하기 위한 강력한 구성을 제공합니다.
생성기는 일부 회로 매개변수를 사용하여 회로 기술을 생성하는 프로그램입니다.
이 섹션에서는 chisel 생성기가 매개변수를 가져오는 방법에 대해 논의하는 것으로 시작하겠습니다.

<span style="color:blue">**Example: Parameterized Scala Object**</span><br>
모든 Chisel `Module`은 다른 것과 마찬가지로 Scala 클래스입니다.
Scala 클래스는 다음과 같이 매개변수화될 수 있음을 기억하십시오.

In [8]:
class ParameterizedScalaObject(param1: Int, param2: String) {
  println(s"I have parameters: param1 = $param1 and param2 = $param2")
}
val obj1 = new ParameterizedScalaObject(4,     "Hello")
val obj2 = new ParameterizedScalaObject(4 + 2, "World")

I have parameters: param1 = 4 and param2 = Hello
I have parameters: param1 = 6 and param2 = World


defined class ParameterizedScalaObject
obj1: ParameterizedScalaObject = ammonite.$sess.cmd7$Helper$ParameterizedScalaObject@2ceb0d2e
obj2: ParameterizedScalaObject = ammonite.$sess.cmd7$Helper$ParameterizedScalaObject@3dad1e71

<span style="color:blue">**Example: Parameterized Chisel Object**</span><br>
Chisel 모듈은 같은 방식으로 매개변수화할 수 있습니다. 다음 모듈에는 모든 입력 및 출력의 너비에 대한 매개변수가 있습니다. 코드 블록을 실행하면 생성된 Verilog가 인쇄됩니다. 매개변수를 사용하여 새 매개변수를 반영하도록 출력이 변경되는지 확인합니다.

In [9]:
class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int) extends Module {
  require(in0Width >= 0)
  require(in1Width >= 0)
  require(sumWidth >= 0)
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  // a +& b includes the carry, a + b does not
  io.sum := io.in0 +& io.in1
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 6)))

Elaborating design...
Done elaborating.
module ParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [5:0] io_sum
);
  wire [3:0] _GEN_0 = {{3'd0}, io_in0}; // @[cmd8.sc 11:20]
  wire [4:0] _T = _GEN_0 + io_in1; // @[cmd8.sc 11:20]
  assign io_sum = {{1'd0}, _T}; // @[cmd8.sc 11:20]
endmodule



defined class ParameterizedWidthAdder

위의 코드 블록에는 `require(...)` 문이 있습니다. 이것은 생성기가 특정 매개변수화로만 작동하거나 일부 매개변수화가 상호 배타적이거나 무의미할 때 유용한 사전 정교화 주장입니다. 위의 코드 블록은 너비가 음수가 아닌지 확인합니다.

`assert(...)`라는 simulation-time assertions을 위한 별도의 구성이 있습니다.

## Sorting with Parameterized Modules
다음 코드 블록은 모듈 2.3의 'Sort4'와 유사한 매개변수화된 정렬입니다. 매개변수화된 너비 IO가 있는 가산기의 이전 예와 달리 이 예에는 고정 IO가 있습니다. 매개변수는 모듈 내부에서 생성되는 하드웨어를 제어합니다.
![Sort4](images/Sorter4.png)
<span style="color:blue">**Example: Parameterized 4-Input Sort**</span><br>
2.3과 달리 이 구현은 내림차순 또는 오름차순 정렬로 매개변수화됩니다.

In [10]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4(ascending: Boolean) extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })
    
  // this comparison funtion decides < or > based on the module's parameterization
  def comp(l: UInt, r: UInt): Bool = {
      if (ascending) {
        l < r
      } else {
        l > r
    }
  }

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(comp(io.in0, io.in1)) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(comp(io.in2, io.in3)) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(comp(row11, row12)) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(comp(row10, row13)) {
    row20 := row10            // preserve the first and the forth elements
    row23 := row13
  }.otherwise {
    row20 := row13            // swap the first and the forth elements
    row23 := row10
  }

  when(comp(row20, row21)) {
    io.out0 := row20            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row20
  }

  when(comp(row22, row23)) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // swap first two elements
    io.out3 := row22
  }
}



// Here are the testers
test(new Sort4(true)) { c => 
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(3.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(9.U)
  c.io.out3.expect(12.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)
}
test(new Sort4(false)) { c =>
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(12.U)
  c.io.out1.expect(9.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(3.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(13.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(4.U)
  c.io.out3.expect(1.U)

  c.io.in0.poke(1.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(13.U)
  c.io.out0.expect(13.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(4.U)
  c.io.out3.expect(1.U)
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test Sort4 Success: 0 tests passed in 2 cycles in 0.020738 seconds 96.44 Hz
Elaborating design...
Done elaborating.
test Sort4 Success: 0 tests passed in 2 cycles in 0.004749 seconds 421.10 Hz
SUCCESS!!


defined class Sort4

---
# Option and Default Arguments

함수가 값을 반환할 때도 있고 반환하지 않을 때도 있습니다. 값을 반환할 수 없을 때 오류가 발생하는 대신 Scala에는 유형 시스템에서 이를 인코딩하는 메커니즘이 있습니다.

<span style="color:blue">**Example: Erroneous Map Index Call**</span><br>
다음 예에는 여러 key/value 쌍이 포함된 맵이 있습니다. 누락된 key/value 쌍에 액세스하려고 하면 런타임 오류가 발생합니다.

In [14]:
val map = Map("a" -> 1)
val a = map("a")
println(a)
val b = map("b")
println(b)

1


: 

<span style="color:blue">**Example: Getting Uncertain Indices**</span><br>
그러나 'Map'은 **get** 메서드를 통해 키 값에 액세스하는 또 다른 방법을 제공합니다. 이것을 사용하면 추상 클래스 `Option`의 값을 반환합니다. `Option`에는 `Some`과 `None`의 두 가지 하위 클래스가 있습니다.

In [15]:
val map = Map("a" -> 1)
val a = map.get("a")
println(a)
val b = map.get("b")
println(b)

Some(1)
None


map: Map[String, Int] = Map("a" -> 1)
a: Option[Int] = Some(1)
b: Option[Int] = None

이후 섹션에서 볼 수 있듯이 `Option`은 사용자가 일치 문을 사용하여 Scala 유형 및 값을 확인할 수 있도록 하기 때문에 매우 중요합니다.

<span style="color:blue">**Example: Get Or Else!**</span><br>
`Map`과 마찬가지로 `Option`에는 `None`에서 호출하면 오류가 발생하는 `get` 메서드도 있습니다. 이러한 경우 **`getOrElse`**를 사용하여 기본값을 제공할 수 있습니다.

In [22]:
val some = Some(1)
val none = None
println(some.get)          // Returns 1
//println(none.get)        // Errors!
println(some.getOrElse(2)) // Returns 1
println(none.getOrElse(2)) // Returns 2

3
3


some: None.type = None
none: None.type = None

## Options for Parameters with Defaults

객체나 함수에 많은 매개변수가 있는 경우 항상 매개변수를 완전히 지정하는 것은 지루하고 오류가 발생하기 쉽습니다. 모듈 1에서 명명된 인수와 매개변수 기본값을 소개했습니다. 때로는 매개변수에 좋은 기본값이 없습니다. 이러한 상황에서 `Option`은 기본값 `None`과 함께 사용할 수 있습니다.

<span style="color:blue">**Example: Optional Reset**</span><br>
다음은 입력을 한 클럭 주기만큼 지연시키는 블록을 보여줍니다. 기본값인 `resetValue = None`이면 레지스터는 재설정 값이 없고 가비지로 초기화됩니다.
이것은 이 레지스터가 리셋되지 않았음을 나타내기 위해 리셋 값으로 -1을 사용하는 것과 같이 "없음"을 나타내기 위해 정상 범위 밖의 값을 사용하는 일반적이지만 추악한 경우를 피합니다.

In [17]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
    val io = IO(new Bundle {
        val in  = Input( UInt(16.W))
        val out = Output(UInt(16.W))
    })
    val reg = if (resetValue.isDefined) { // resetValue = Some(number)
        RegInit(resetValue.get)
    } else { //resetValue = None
        Reg(UInt())
    }
    reg := io.in
    io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

Elaborating design...
Done elaborating.
module DelayBy1(
  input         clock,
  input         reset,
  input  [15:0] io_in,
  output [15:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [15:0] reg_; // @[cmd16.sc 9:12]
  assign io_out = reg_; // @[cmd16.sc 12:12]
  always @(posedge clock) begin
    reg_ <= io_in; // @[cmd16.sc 11:9]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `el

defined class DelayBy1

---
# Match/Case Statements
Scala *matching* 개념은 Chisel 전체에서 사용되며 모든 Chisel 프로그래머의 기본 이해의 일부가 되어야 합니다. Scala는 다음을 지원하는 일치 연산자를 제공합니다.
- C *switch* 문과 같은 대안에 대한 간단한 테스트
- 값의 임시 조합에 대한 보다 복잡한 테스트
- 변수의 유형을 알 수 없거나 지정되지 않은 경우(예:
   - 변수는 이기종 목록에서 가져옵니다. ```val mixedList = List(1, "string", false)```
   - 또는 변수가 상위 클래스의 구성원인 것으로 알려져 있지만 어떤 특정 하위 클래스인지는 알 수 없습니다.
- *정규식*으로 지정된 문자열의 하위 문자열 추출


<span style="color:blue">**Example: Value Matching**</span><br>
다음 예에서는 **일치**하는 변수의 **value**에 따라 다른 **case** 문을 실행합니다.

In [23]:
// y is an integer variable defined somewhere else in the code
val y = 2
/// ...
val x = y match {
  case 0 => "zero" // One common syntax, preferred if fits in one line
  case 1 =>        // Another common syntax, preferred if does not fit in one line.
      "one"        // Note the code block continues until the next case
  case 2 => {      // Another syntax, but curly braces are not required
      "two"
  }
  case _ => "many" // _ is a wildcard that matches all values
}
println("y is " + x)

y is two


y: Int = 2
x: String = "two"

일치 연산자는 가능한 값을 확인하고 각 경우에 대해 문자열을 반환합니다. 주의해야 할 몇 가지 사항:
- ```=>``` 연산자 다음에 오는 각 코드 블록은 일치의 끝 중괄호 또는 다음 case 문에 도달할 때까지 계속됩니다.
- case 문이 일치하는 순서대로 일치하는 항목을 검색하며, case 문이 일치하면 다른 항목이 일치하지 않습니다.
다른 case 문에 대한 검사가 수행됩니다.
- 찾을 수 없는 값을 처리하기 위해 밑줄을 와일드카드로 사용합니다.

<span style="color:blue">**Example: Multiple Value Matching**</span><br>
또한 여러 변수를 동시에 일치시킬 수 있습니다. 다음은 match 문과 값의 튜플로 구현된 진리표의 간단한 예입니다.

In [19]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}
println(animalType(true, true))

wolverine


defined function animalType

<span style="color:blue">**Example: Type Matching**</span><br>
Scala는 강력한 형식의 언어이므로 모든 개체의 형식은 런타임 중에 알려집니다. **match 문**을 사용하여 이 유형 정보를 사용하여 제어 흐름을 지시할 수 있습니다.

In [37]:
val sequence = Seq("a", 3.U(2.W), 0.0)
sequence.foreach { x =>
  x match {
    case s: String => println(s"$x is a String")
    case s: Int    => println(s"$x is an Int")
    case s: Double => println(s"$x is a Double")
    case _ => println(s"$x is an unknown type!")
  }
}

a is a String
UInt<2>(3) is an unknown type!
0.0 is a Double


sequence: Seq[Any] = List("a", UInt<2>(3), 0.0)

<span style="color:blue">**Example: Multiple Type Matching**</span><br>
값에 여러 유형 중 하나가 있는지 여부를 일치시키려면 다음 구문을 사용하십시오. *일치할 때 **반드시** `_`를 사용해야 합니다.*

In [38]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case _: Int | _: Double => println(s"$x is a number!")
    case _ => println(s"$x is an unknown type!")
  }
}

a is an unknown type!
1 is a number!
0.0 is a number!


sequence: Seq[Any] = List("a", 1, 0.0)

<span style="color:blue">**Example: Type Matching and Erasure**</span><br>
유형 일치에는 몇 가지 제한 사항이 있습니다. Scala는 JVM에서 실행되고 JVM은 다형성 유형을 유지하지 않기 때문에 런타임에 일치할 수 없습니다(모두 지워지기 때문에). 다음 예는 `[String]`, `[Int]`, `[Double]` 다형성 유형이 지워지고 case 문은 **실제로** 일치하는 첫 번째 case 문과 항상 일치합니다.  `Seq`.

In [22]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
  x match {
    case s: Seq[String] => println(s"$x is a String")
    case s: Seq[Int]    => println(s"$x is an Int")
    case s: Seq[Double] => println(s"$x is a Double")
  }
}

List(a) is a String
List(1) is a String
List(0.0) is a String


sequence: Seq[Seq[Any]] = List(List("a"), List(1), List(0.0))

스칼라 컴파일러는 위의 예와 같은 코드를 구현하면 일반적으로 경고를 표시합니다.다음 코드 블록은 'if/else' 대신 match 구문을 사용하는 동일한 'DelayBy1' 모듈을 보여줍니다.

<span style="color:blue">**Example: Optional Reset Matching**</span><br>
다음 코드 블록은 `if/else` 대신 match 구문을 사용하는 동일한 `DelayBy1` 모듈을 보여줍니다.

In [23]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
  val io = IO(new Bundle {
    val in  = Input( UInt(16.W))
    val out = Output(UInt(16.W))
  })
  val reg = resetValue match {
    case Some(r) => RegInit(r)
    case None    => Reg(UInt())
  }
  reg := io.i
  io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

Elaborating design...
Done elaborating.
module DelayBy1(
  input         clock,
  input         reset,
  input  [15:0] io_in,
  output [15:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [15:0] reg_; // @[cmd22.sc 8:24]
  assign io_out = reg_; // @[cmd22.sc 11:10]
  always @(posedge clock) begin
    reg_ <= io_in; // @[cmd22.sc 10:7]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `el

defined class DelayBy1

---
# IOs with Optional Fields

때로는 IO가 선택적으로 포함되거나 제외되기를 원합니다. 디버깅을 위해 볼 수 있는 좋은 내부 상태가 있을 수 있지만 시스템에서 생성기를 사용할 때 숨기고 싶을 수 있습니다. 적절한 기본값이 있기 때문에 생성기에 모든 상황에서 연결할 필요가 없는 일부 입력이 있을 수 있습니다.

<span style="color:blue">**Example: Optional IO with Option**</span><br>
선택적 번들 필드는 이 기능을 얻는 한 가지 방법입니다. 다음 예에서는 선택적으로 캐리를 받는 1비트 가산기를 보여줍니다. 캐리가 포함된 경우 `io.carryIn`은 `Some[UInt]` 유형을 가지며 IO 번들에 포함됩니다.
캐리가 포함되지 않은 경우 `io.carryIn`은 `None` 유형을 가지며 IO 번들에서 제외됩니다.

In [24]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = if (hasCarry) Some(Input(UInt(1.W))) else None
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn.getOrElse(0.U)
  io.s := sum(0)
  io.carryOut := sum(1)
}

test(new HalfFullAdder(false)) { c =>
  require(!c.hasCarry, "DUT must be half adder")
  // 0 + 0 = 0
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(0.U)
  // 0 + 1 = 1
  c.io.b.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 1 + 1 = 2
  c.io.a.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 1 + 0 = 1
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
}

test(new HalfFullAdder(true)) { c =>
  require(c.hasCarry, "DUT must be half adder")
  c.io.carryIn.get.poke(0.U)
  // 0 + 0 + 0 = 0
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(0.U)
  // 0 + 0 + 1 = 1
  c.io.b.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 0 + 1 + 1 = 2
  c.io.a.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 0 + 1 + 0 = 1
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)

  c.io.carryIn.get.poke(1.U)
  // 1 + 0 + 0 = 1
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 1 + 0 + 1 = 2
  c.io.b.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 1 + 1 + 1 = 3
  c.io.a.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(1.U)
  // 1 + 1 + 0 = 2
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

Elaborating design...
Done elaborating.
test HalfFullAdder Success: 0 tests passed in 2 cycles in 0.002117 seconds 944.92 Hz
Elaborating design...
Done elaborating.
test HalfFullAdder Success: 0 tests passed in 2 cycles in 0.003014 seconds 663.57 Hz
SUCCESS!!


defined class HalfFullAdder

<span style="color:blue">**Example: Optional IO with Zero-Width Wires**</span><br>
`Option`과 유사한 기능을 구현하는 또 다른 방법은 너비가 0인 와이어를 사용하는 것입니다. CHISEL 유형은 너비가 0일 수 있습니다. 너비가 0인 IO는 방출된 Verilog에서 제거되고 너비가 0인 와이어의 값을 사용하려고 하는 모든 것은 상수 0을 얻습니다. 0이 합리적인 기본값이라면 너비가 0인 와이어는 옵션에서 일치하거나 `getOrElse`를 호출할 필요가 없기 때문에 유용할 수 있습니다.

In [25]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = Input(if (hasCarry) UInt(1.W) else UInt(0.W))
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn
  io.s := sum(0)
  io.carryOut := sum(1)
}
println("Half Adder:")
println(getVerilog(new HalfFullAdder(false)))
println("\n\nFull Adder:")
println(getVerilog(new HalfFullAdder(true)))

Half Adder:
Elaborating design...
Done elaborating.
module HalfFullAdder(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_s,
  output  io_carryOut
);
  wire [1:0] _T = io_a + io_b; // @[cmd24.sc 9:18]
  wire [2:0] sum = {{1'd0}, _T}; // @[cmd24.sc 9:26]
  assign io_s = sum[0]; // @[cmd24.sc 10:14]
  assign io_carryOut = sum[1]; // @[cmd24.sc 11:21]
endmodule



Full Adder:
Elaborating design...
Done elaborating.
module HalfFullAdder(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  input   io_carryIn,
  output  io_s,
  output  io_carryOut
);
  wire [1:0] _T = io_a + io_b; // @[cmd24.sc 9:18]
  wire [1:0] _GEN_0 = {{1'd0}, io_carryIn}; // @[cmd24.sc 9:26]
  wire [2:0] sum = _T + _GEN_0; // @[cmd24.sc 9:26]
  assign io_s = sum[0]; // @[cmd24.sc 10:14]
  assign io_carryOut = sum[1]; // @[cmd24.sc 11:21]
endmodule



defined class HalfFullAdder

---
# Implicits
많은 상용구 코드가 필요한 프로그래밍을 하는 경우가 종종 있습니다. 이 사용 사례를 처리하기 위해 스칼라는 **암시적** 개념을 도입했는데, 이를 통해 컴파일러가 구문상의 설탕을 처리할 수 있습니다. 많은 일들이 뒤에서 일어나기 때문에 암시는 매우 마술처럼 보일 수 있습니다. 이 섹션에서는 몇 가지 기본 예제를 분석하여 해당 예제가 무엇이며 일반적으로 사용되는 위치를 설명합니다.

## Implicit Arguments
때때로 코드는 일련의 함수 호출 내 깊숙한 곳에서 일종의 최상위 변수에 액세스해야 합니다. 모든 함수 호출을 통해 이 변수를 수동으로 스레딩하는 대신 암시적 인수를 사용하여 이를 수행할 수 있습니다.

<span style="color:blue">**Example: Implicit Cats**</span><br>
다음 예에서는 암시적 또는 명시적으로 고양이 수를 전달할 수 있습니다.

In [56]:
object CatDog {
  implicit val numberOfCats: Int = 3
  //implicit val numberOfDogs: Int = 5

  def tooManyCats(nDogs: Int)(implicit nCats: Int): Boolean = nCats > nDogs
    
  val imp = tooManyCats(2)    // Argument passed implicitly!
  val exp = tooManyCats(2)(1) // Argument passed explicitly!
}
CatDog.imp
CatDog.exp

defined object CatDog
res55_1: Boolean = true
res55_2: Boolean = false

무슨 일이야? 먼저 암시적 값 **numberOfCats**를 정의합니다. 주어진 범위에서 **주어진 유형의 암시적 값은 하나만 있을 수 있습니다**. 그런 다음 두 개의 인수 목록을 사용하는 함수를 정의합니다. 첫 번째는 모든 명시적 매개변수이고 두 번째는 모든 암시적 매개변수입니다. **tooManyCats**를 호출할 때 두 번째 암시적 인수 목록을 생략하거나(컴파일러가 자동으로 찾을 수 있도록 함) 인수를 명시적으로 제공합니다(암시적 값과 다를 수 있음).

다음은 암시적 인수가 *실패*할 수 있는 방법입니다.
- 주어진 유형의 두 개 이상의 암시적 값이 범위에 정의되어 있습니다.
- 컴파일러가 함수 호출에 필요한 암시적 값을 찾을 수 없는 경우

<span style="color:blue">**Example: Implicit Logging**</span><br>
다음 코드 블록은 암시적 인수를 사용하여 치즐 생성기에서 logging을 구현하는 방법을 보여줍니다.

***참고: Scala에 logging하는 더 좋은 방법이 있습니다!***

In [27]:
sealed trait Verbosity
implicit case object Silent extends Verbosity
case object Verbose extends Verbosity

class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int)(implicit verbosity: Verbosity)
extends Module {
  def log(msg: => String): Unit = verbosity match {
    case Silent =>
    case Verbose => println(msg)
  }
  require(in0Width >= 0)
  log(s"in0Width of $in0Width OK")
  require(in1Width >= 0)
  log(s"in1Width of $in1Width OK")
  require(sumWidth >= 0)
  log(s"sumWidth of $sumWidth OK")
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  log("Made IO")
  io.sum := io.in0 + io.in1
  log("Assigned output")
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)))
println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)(Verbose)))

Elaborating design...
Done elaborating.
module ParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [4:0] io_sum
);
  wire [3:0] _GEN_0 = {{3'd0}, io_in0}; // @[cmd26.sc 23:20]
  wire [3:0] _T_1 = _GEN_0 + io_in1; // @[cmd26.sc 23:20]
  assign io_sum = {{1'd0}, _T_1}; // @[cmd26.sc 23:20]
endmodule

Elaborating design...
in0Width of 1 OK
in1Width of 4 OK
sumWidth of 5 OK
Made IO
Assigned output
Done elaborating.
module ParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [4:0] io_sum
);
  wire [3:0] _GEN_0 = {{3'd0}, io_in0}; // @[cmd26.sc 23:20]
  wire [3:0] _T_1 = _GEN_0 + io_in1; // @[cmd26.sc 23:20]
  assign io_sum = {{1'd0}, _T_1}; // @[cmd26.sc 23:20]
endmodule



defined trait Verbosity
defined object Silent
defined object Verbose
defined class ParameterizedWidthAdder

## Implicit Conversions
암시적 인수와 마찬가지로 암시적 함수(**암시적 변환**이라고도 함)는 상용구 코드를 줄이는 데 사용됩니다. 보다 구체적으로, 하나의 Scala 객체를 다른 객체로 자동 변환하는 데 사용됩니다.

<span style="color:blue">**Example: Implicit Conversion**</span><br>
다음 예에는 `Animal`과 `Human`의 두 가지 클래스가 있습니다. '동물'에는 '종' 필드가 있지만 '인간'에는 없습니다. 그러나 암시적 변환을 구현하면 '인간'에서 '종'을 호출할 수 있습니다.

In [28]:
class Animal(val name: String, val species: String)
class Human(val name: String)
implicit def human2animal(h: Human): Animal = new Animal(h.name, "Homo sapiens")
val me = new Human("Adam")
println(me.species)

Homo sapiens


defined class Animal
defined class Human
defined function human2animal
me: Human = ammonite.$sess.cmd27$Helper$Human@61db59c7

일반적으로 implicits은 코드를 혼란스럽게 만들 수 있으므로 최후의 수단으로 사용하는 것이 좋습니다. 먼저 inheritance, traits 또는 method overloading을 시도하십시오.

---
# Generator Example
다음 예는 1비트 입력 Mealy 머신용 생성기를 보여줍니다.
[Wikipedia](https://en.wikipedia.org/wiki/Mealy_machine#/media/File:Mealy.png)의 예제를 기반으로 한 테스트가 있습니다.
코드를 읽고 무슨 일이 일어나는지 따라해보세요.

<span style="color:blue">**Example: Mealy Machine**</span><br>
Mealy 머신 생성기의 매개변수화를 직접 만들고 아래 코드 블록에 테스트를 작성해 보세요.

In [3]:
// Mealy machine has
case class BinaryMealyParams(
  // number of states
  nStates: Int,
  // initial state
  s0: Int,
  // function describing state transition
  stateTransition: (Int, Boolean) => Int,
  // function describing output
  output: (Int, Boolean) => Int
) {
  require(nStates >= 0)
  require(s0 < nStates && s0 >= 0)
}

class BinaryMealy(val mp: BinaryMealyParams) extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt())
  })

  val state = RegInit(UInt(), mp.s0.U)

  // output zero if no states
  io.out := 0.U
  for (i <- 0 until mp.nStates) {
    when (state === i.U) {
      when (io.in) {
        state  := mp.stateTransition(i, true).U
        io.out := mp.output(i, true).U
      }.otherwise {
        state  := mp.stateTransition(i, false).U
        io.out := mp.output(i, false).U
      }
    }
  }
}

// example from https://en.wikipedia.org/wiki/Mealy_machine
val nStates = 3
val s0 = 2
def stateTransition(state: Int, in: Boolean): Int = {
  if (in) {
    1
  } else {
    0
  }
}
def output(state: Int, in: Boolean): Int = {
  if (state == 2) {
    return 0
  }
  if ((state == 1 && !in) || (state == 0 && in)) {
    return 1
  } else {
    return 0
  }
}

val testParams = BinaryMealyParams(nStates, s0, stateTransition, output)

test(new BinaryMealy(testParams)) { c =>
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

cmd3.sc:26: value === is not a member of Int
    when (state === state.U) {
                ^cmd3.sc:28: value := is not a member of Int
  Expression does not convert to assignment because receiver is not assignable.
        state  := mp.stateTransition(state, true).U
               ^cmd3.sc:31: value := is not a member of Int
  Expression does not convert to assignment because receiver is not assignable.
        state  := mp.stateTransition(state, false).U
               ^Compilation Failed

: 

---
# You're done!

[Return to the top.](#top)